In [1]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from config_profile import args
from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from Utils import L2Norm, cv2_scale, np_reshape

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
from descriptor_CNN import DesNet
model = DesNet()
model.eval()
if args.cuda:
    model.cuda()
weight_path = "checkpoint.pth"
trained_weight = torch.load(weight_path)
model.load_state_dict(trained_weight['state_dict'])

<All keys matched successfully>

In [6]:
# parameters setting
query_path = "Stereo_images/p_image/"
image_path = "Stereo_images/q_image/"
query_num = 5    # total query image number
image_num = 5    # total retrieval image number
kps_num = 50       # key points number on one image
patch_size = 32    # patch size
patches = torch.zeros(query_num+image_num, kps_num,1, 32, 32)
keypoints = torch.zeros(query_num+image_num, kps_num, 2)

In [7]:
# load patches
# query
patches_p_dir = "../huangtin_hw3/p_patches.pth"
patches_p = torch.load(patches_p_dir)
print("query patches: ", patches_p.shape)
patches_p =  patches_p.view(-1, 1, 32, 32).cuda()

# dataset
patches_q_dir = "../huangtin_hw3/q_patches.pth"
patches_q = torch.load(patches_q_dir)
print("dataset patches: ", patches_q.shape)
patches_q =  patches_q.view(-1, 1, 32, 32).cuda()

query patches:  torch.Size([5, 50, 1, 32, 32])
dataset patches:  torch.Size([5, 50, 1, 32, 32])


In [9]:
# query
description_p = model(patches_p)
description_p = description_p.view(-1, 50, 128)

# dataset
description_q = model(patches_q)
description_q = description_q.view(-1, 50, 128)

In [10]:
print("p description: ", description_p.shape)
print("q description: ", description_q.shape)

p description:  torch.Size([5, 50, 128])
q description:  torch.Size([5, 50, 128])


In [11]:
torch.save(description_p, "p_description.pth")
torch.save(description_q, "q_description.pth")

In [12]:
description_p

tensor([[[ 0.0558, -0.0223, -0.0339,  ..., -0.0890,  0.1579, -0.1363],
         [-0.0798, -0.1162, -0.1483,  ...,  0.0696,  0.0472, -0.0623],
         [-0.0324,  0.0574, -0.0549,  ...,  0.0905,  0.0009, -0.0134],
         ...,
         [-0.0893, -0.0906,  0.0098,  ..., -0.0319,  0.0208, -0.0527],
         [ 0.0854,  0.0371, -0.1394,  ..., -0.0092,  0.1534, -0.0751],
         [ 0.0684,  0.1484,  0.1038,  ..., -0.0082, -0.1399, -0.0932]],

        [[-0.0658, -0.0169, -0.0219,  ..., -0.2698, -0.0608,  0.0515],
         [-0.1007, -0.0641,  0.0131,  ...,  0.1352,  0.0281, -0.0673],
         [ 0.0819,  0.1091,  0.1318,  ...,  0.1033, -0.0431, -0.0120],
         ...,
         [ 0.0865, -0.0641, -0.2936,  ...,  0.0046, -0.0300, -0.0850],
         [ 0.0494,  0.0693, -0.0622,  ...,  0.0542,  0.0267, -0.0751],
         [-0.0557,  0.1777,  0.0126,  ..., -0.0003, -0.1812,  0.1345]],

        [[-0.0472,  0.0028, -0.0238,  ...,  0.1323, -0.1395,  0.1121],
         [-0.0777, -0.1367,  0.0401,  ..., -0